In [1]:
import numpy as np
import pandas as pd
import math
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.pylab as pylab
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

%matplotlib inline

In [2]:
bl = pd.read_excel('bank_personal_loan.xlsx')
bl.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,10,34,9,180,93023,1,8.9,3,0,1,0,0,0,0
1,17,38,14,130,95010,4,4.7,3,134,1,0,0,0,0
2,19,46,21,193,91604,2,8.1,3,0,1,0,0,0,0
3,30,38,13,119,94104,1,3.3,2,0,1,0,1,1,1
4,39,42,18,141,94114,3,5.0,3,0,1,1,1,1,0


In [3]:
bl.columns = bl.columns.str.lower()
bl.columns = bl.columns.replace((' ','_'), inplace=True)

AttributeError: 'Index' object has no attribute 'replace'

In [4]:
bl.head()

,id,age,experience,income,zip code,family,ccavg,education,mortgage,personal loan,securities account,cd account,online,creditcard
0,10,34,9,180,93023,1,8.9,3,0,1,0,0,0,0
1,17,38,14,130,95010,4,4.7,3,134,1,0,0,0,0
2,19,46,21,193,91604,2,8.1,3,0,1,0,0,0,0
3,30,38,13,119,94104,1,3.3,2,0,1,0,1,1,1
4,39,42,18,141,94114,3,5.0,3,0,1,1,1,1,0


In [5]:
bl.shape

(480, 14)

In [6]:
bl.isnull().sum()

id                    0
age                   0
experience            0
income                0
zip code              0
family                0
ccavg                 0
education             0
mortgage              0
personal loan         0
securities account    0
cd account            0
online                0
creditcard            0
dtype: int64

In [7]:
bl.describe().T

,count,mean,std,min,25%,50%,75%,max
id,480.0,2390.650000,1394.393674,10.0,1166.50,2342.0,3566.00,4981.0
age,480.0,45.066667,11.590964,26.0,35.00,45.0,55.00,65.0
experience,480.0,19.843750,11.582443,0.0,9.00,20.0,30.00,41.0
income,480.0,144.745833,31.584429,60.0,122.00,142.5,172.00,203.0
zip code,480.0,93153.202083,1759.223753,90016.0,91908.75,93407.0,94705.50,96008.0
family,480.0,2.612500,1.115393,1.0,2.00,3.0,4.00,4.0
ccavg,480.0,3.905347,2.097679,0.0,2.60,3.8,5.35,10.0
education,480.0,2.233333,0.753373,1.0,2.00,2.0,3.00,3.0
mortgage,480.0,100.845833,160.847862,0.0,0.00,0.0,192.50,617.0
personal loan,480.0,1.000000,0.000000,1.0,1.00,1.0,1.00,1.0


In [8]:
bl.corr()

,id,age,experience,income,zip code,family,ccavg,education,mortgage,personal loan,securities account,cd account,online,creditcard
id,1.000000,-0.067018,-0.070878,-0.023495,-0.003576,-0.012886,-0.041163,0.040625,0.030496,NaN,-0.016317,-0.030523,-0.041305,0.053741
age,-0.067018,1.000000,0.994455,0.021260,-0.030308,-0.001712,-0.002502,-0.009196,0.041622,NaN,-0.008705,0.053308,0.028577,0.026542
experience,-0.070878,0.994455,1.000000,0.013587,-0.034159,0.005161,-0.019902,-0.020217,0.038025,NaN,-0.005240,0.059372,0.032235,0.032025
income,-0.023495,0.021260,0.013587,1.000000,0.027232,-0.020876,0.214983,0.122170,0.189672,NaN,-0.053058,0.069669,0.075136,0.013477
zip code,-0.003576,-0.030308,-0.034159,0.027232,1.000000,0.009322,0.096757,0.005492,-0.015616,NaN,0.023399,0.013037,-0.042927,0.069077
family,-0.012886,-0.001712,0.005161,-0.020876,0.009322,1.000000,0.005408,-0.239996,0.070556,NaN,-0.106005,-0.003085,-0.039178,0.050746
ccavg,-0.041163,-0.002502,-0.019902,0.214983,0.096757,0.005408,1.000000,0.077965,0.023250,NaN,0.017323,0.037370,-0.017309,0.025837
education,0.040625,-0.009196,-0.020217,0.122170,0.005492,-0.239996,0.077965,1.000000,-0.026044,NaN,-0.041852,-0.071054,0.000567,-0.038536
mortgage,0.030496,0.041622,0.038025,0.189672,-0.015616,0.070556,0.023250,-0.026044,1.000000,NaN,-0.016809,0.084738,0.063899,0.039181
personal loan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
